In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip3 install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 36.6 MB/s eta 0:00:0000:0100:01


In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch import nn 
import mediapipe as mp

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 112 * 112, 512),
            nn.Linear(512,28),
            #nn.Softmax(dim=1)

        )

    def forward(self,x):
        return self.model(x)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN()  # Ensure the CNN class is defined in the notebook
model.load_state_dict(torch.load('model.pth', map_location=device))
model.to(device)

cross = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.eval()

C:\Users\Gpentela\AppData\Local\Temp\ipykernel_14156\3445487083.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth', map_locatio

CNN(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=802816, out_features=512, bias=True)
    (9): Linear(in_features=512, out_features=28, bias=True)
  )
)

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# ASL Labels
label_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
              'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 
              'W', 'X', 'Y', 'Z']

print("Label list loaded:", label_list)

# Open webcam
cap = cv2.VideoCapture(0)



# Define transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def detect_sign():
    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.flip(frame_rgb, 1)  
            frame_rgb.flags.writeable = False
            results = hands.process(frame_rgb)
            frame_rgb.flags.writeable = True
            frame_rgb = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Get bounding box around hand
                    h, w, _ = frame.shape
                    x_min = min([landmark.x for landmark in hand_landmarks.landmark]) * w
                    y_min = min([landmark.y for landmark in hand_landmarks.landmark]) * h
                    x_max = max([landmark.x for landmark in hand_landmarks.landmark]) * w
                    y_max = max([landmark.y for landmark in hand_landmarks.landmark]) * h

                    # Expand bounding box slightly
                    margin = 20
                    x_min = max(int(x_min - margin), 0)
                    y_min = max(int(y_min - margin), 0)
                    x_max = min(int(x_max + margin), w)
                    y_max = min(int(y_max + margin), h)

                    # Extract hand region
                    hand_roi = frame[y_min:y_max, x_min:x_max]

                    if hand_roi.size > 0:  
                        input_image = transform(hand_roi)  # Apply transforms
                        input_tensor = input_image.unsqueeze(0)  # Add batch dimension

                        # Model prediction
                        with torch.no_grad():
                            output = model(input_tensor)  
                            predicted_label_index = torch.argmax(output, dim=1).item()

                        if predicted_label_index < len(label_list):
                            sign_name = label_list[predicted_label_index]
                        else:
                            sign_name = "Unknown Sign"  

                        # Display prediction
                        cv2.putText(frame_rgb, sign_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        cv2.rectangle(frame_rgb, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            cv2.imshow('ASL Detection', frame_rgb)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_sign()

Captured Images and Predictions:
Image 1: Predicted Label = P
